In [1]:
pip install ultralytics torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.6/876.6 kB 38.2 MB/s eta 0:00:00


In [2]:
import cv2
import torch
from ultralytics import YOLO
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [9]:
# Loading the best model
model = models.resnet18(weights='ResNet18_Weights.IMAGENET1K_V1')
model.fc = nn.Linear(model.fc.in_features, 6) # 6 is no. of classes
model.load_state_dict(torch.load('/content/drive/MyDrive/Robo/freshness_detection_model.pth'))  # Load the best saved model
# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()  # Set the model to evaluation mode

# Now you can use the model for inference on new images
def predict_image(image):
    image = image.to(device)
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)
    return predicted

def load_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

classes = ['freshapples', 'freshbanana', 'freshorange', 'rottenapples', 'rottenbanana', 'rottenoranges']

# Usage
def check_freshness(front_image, back_image, name):
  front_image = load_image(front_image)
  back_image = load_image(back_image)
  predicted_class1 = predict_image(front_image)
  predicted_class2 = predict_image(back_image)

  if 'fresh' in classes[int(predicted_class1)] and 'fresh' in classes[int(predicted_class2)]:
    print("Result : Fresh")
    return name, "fresh"
  else:
    print("Result : Not Fresh")
    return name, "not fresh"


In [11]:

# Load the YOLOv8 model (ensure the ultralytics package is installed)
yolo = YOLO('/content/drive/MyDrive/Robo/yolov8n.pt')  # Using the smallest model, you can switch to a larger version if needed

# Function to detect the fruit or vegetable and trigger freshness check
def detect_fruit_or_vegetable(front_image_path, back_image_path, images):
    # Detect objects in the front image
    results = yolo(front_image_path, verbose=False)

    # Extract the detected object's name (class label)
    if len(results[0].boxes) > 0:
        detected_item = yolo.names[int(results[0].boxes[0].cls)]

        print(f"Triggering freshness analysis for {detected_item} ...")

        # Call the freshness check function using front and back images
        name, status = check_freshness(images['front'], images['back'], name = detected_item)
        return name, status
    else:
        print("No item detected in the image.")



In [12]:
# List of items with front and back image paths
items = {
    "item3": {"front": "/content/drive/MyDrive/Robo/images/item3_front.jpg", "back": "/content/drive/MyDrive/Robo/images/item3_back.jpg"},
    # Add more items here
}

# Dictionary to store counts of (name, status)
item_counts = {}

# List to store the output in the format ["Item", "Status", "Counts"]
output_list = [["Item", "Status", "Counts"],]

# Process each item
for item_id, images in items.items():
    print(f"\nProcessing {item_id}:")

    # Load the front and back images
    front_image = cv2.imread(images["front"])
    back_image = cv2.imread(images["back"])

    name, status = detect_fruit_or_vegetable(front_image, back_image, images)
    # Create a tuple of (name, status)
    key = (name, status)

    # Check if the combination exists in the dictionary
    if key in item_counts:
        # If exists, increment the count
        item_counts[key] += 1
    else:
        # If not, add with an initial count of 1
        item_counts[key] = 1

    # Update the output list with the current item, status, and count
    output_list.append([name, status, item_counts[key]])

# Print the final output list
print("\nFinal Output:")
if len(output_list) > 1:
    for item in output_list:
        print(item)


Processing item3:
Triggering freshness analysis for apple ...
Result : Fresh

Final Output:
['Item', 'Status', 'Counts']
['apple', 'fresh', 1]
